## 1.Problem Statement

---



* This project is to develop a deep learning-based fire image classification system that can accurately and efficiently classify fire images into different categories. 
* Specifically, the system should be able to identify the type of fire, its intensity, and the objects involved in the fire, such as buildings, trees, or vehicles.



##2. Data Acquisition

The data is collected from the online website

##3.Import the libraries and dataset

In [ ]:
import pandas as pd
import os
import cv2
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 4 Split the data into training set and testing set

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

* ImageDataGenerator is a class in the tensorflow.keras.preprocessing.image module of TensorFlow that generates batches of tensor image data with real-time data augmentation.
* The ImageDataGenerator class provides a powerful way to augment and preprocess image data for deep learning models, and can help improve their performance and generalization on a wide range of tasks.

In [ ]:
traindata=ImageDataGenerator()

In [ ]:
traindata=traindata.flow_from_directory(directory='/content/drive/MyDrive/Fire detection/Train',target_size=(224,224))

Found 2320 images belonging to 2 classes.


* We're creating an ImageDataGenerator object called train_datagen and setting various augmentation parameters. 
* We're then using the flow_from_directory method to generate augmented image data on the fly and feed it to the model during training.
* The rescale parameter scales the pixel values of the image to a range of 0-1, and target_size sets the size of the images to be generated.

In [ ]:
traindata.image_shape

(224, 224, 3)

In [ ]:
testdata=ImageDataGenerator()
testdata=testdata.flow_from_directory(directory='/content/drive/MyDrive/Fire detection/Test',target_size=(224,224))

Found 331 images belonging to 2 classes.


In [ ]:
testdata.image_shape

(224, 224, 3)

## 5 Deep Neural Network Architecture

In [ ]:
import tensorflow
import keras,os
from keras.models import Sequential
from keras.layers import Flatten,Dense
from tensorflow.keras import Model

#5.1 DenseNet121

In [ ]:
from keras.applications.densenet import DenseNet121
DenseNet121_model=DenseNet121(include_top=False,input_shape=(224,224,3),classes=2)
DenseNet121_model.summary()

29084464/29084464 [==============================] - 1s 0us/step
Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                       

### Transfer Learning Process



* Transfer learning is a machine learning technique where a pre-trained model   on a large dataset is used as the starting point for a new task. 
* Instead of training a model from scratch on a new dataset, transfer learning allows us to leverage the knowledge learned by the pre-trained model on a similar task to improve the performance of the new model.

In [ ]:
for layer in DenseNet121_model.layers:
  layer.trainable= False

In [ ]:
X=Flatten()(DenseNet121_model.output)
prediction=Dense(2,activation='softmax')(X)

In [ ]:
model=Model(inputs=DenseNet121_model.input,outputs=prediction)

* In this example, we load the VGG16 pre-trained model, freeze its weights, and add a new output layer for our new task. 
* We then create a new model and compile it with an appropriate optimizer and loss function. 
* Finally, we train the model on the new dataset using a generator that provides the input images and their labels.

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 zero_padding2d (ZeroPadding2D)  (None, 230, 230, 3)  0          ['input_1[0][0]']                
                                                                                                  
 conv1/conv (Conv2D)            (None, 112, 112, 64  9408        ['zero_padding2d[0][0]']         
                                )                                                                 
                                                                                              

### Model compiling and Training

In [ ]:
model.compile(optimizer='Adam', loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [ ]:
model.fit(traindata,epochs=3,validation_data=testdata,verbose=1)

Epoch 1/3
73/73 [==============================] - 726s 10s/step - loss: 5.2061 - accuracy: 0.8905 - val_loss: 6.3431 - val_accuracy: 0.8640
Epoch 2/3
73/73 [==============================] - 460s 6s/step - loss: 1.0623 - accuracy: 0.9569 - val_loss: 5.4776 - val_accuracy: 0.8761
Epoch 3/3
73/73 [==============================] - 488s 7s/step - loss: 0.7098 - accuracy: 0.9694 - val_loss: 5.8624 - val_accuracy: 0.8882


In [ ]:
model.save('/content/drive/MyDrive/Saved_Models/fireclassDense1.h5')

* The h5 format is a binary file format used to store large amounts of numerical data, such as arrays and tensors,The h5 format is short for HDF5 (Hierarchical Data Format 5)
*The h5 format is commonly used to store pre-trained deep learning models in machine learning frameworks such as Keras, TensorFlow, and PyTorch. 
*These models can be quite large and complex, and the h5 format provides an efficient way to store and load them.

In [ ]:
traindata.class_indices

{'Fire': 0, 'Neutral': 1}

### Model Testing

In [ ]:
from keras.models import load_model
#from keras.preprocessing import image
import keras.utils as image
test_image=image.load_img('/content/drive/MyDrive/Fire detection/Test/Fire/100.jpg',target_size=(224,224))

test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)



model=load_model('/content/drive/MyDrive/Saved_Models/fireclassDense.h5')
result=model.predict(test_image)
result

if result[0][0] ==1:
  prediction='fire_images'
  print(prediction)

elif result[0][1] ==1:
  prediction='neutral_images'
  print(prediction)


1/1 [==============================] - 2s 2s/step


### 5.2 MobileNetV2 

In [ ]:
from keras.applications.mobilenet_v3 import MobileNetV3
MobileNetV3_model=tensorflow.keras.applications.MobileNetV3Large(
    input_shape=(224,224,3),
    alpha=1.0,
    minimalistic=False,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    classes=1000,
    pooling=None,
    dropout_rate=0.2,
    classifier_activation="softmax",
    include_preprocessing=True,
)
MobileNetV3_model.summary()

22661472/22661472 [==============================] - 0s 0us/step
Model: "MobilenetV3large"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                  

###Transfer Learning Process

In [ ]:
for layer in MobileNetV3_model.layers:
  layer.trainable= False


In [ ]:
X=Flatten()(MobileNetV3_model.output)
prediction=Dense(2,activation='softmax')(X)

In [ ]:
model=Model(inputs=MobileNetV3_model.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 Conv (Conv2D)                  (None, 112, 112, 16  432         ['rescaling[0][0]']              
                                )                                                                 
                                                                                            

### Model Compiling and Training

In [ ]:
model.compile(optimizer='Adam', loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [ ]:
model.fit(traindata,epochs=8,validation_data=testdata,verbose=1)

Epoch 1/8
73/73 [==============================] - 129s 2s/step - loss: 0.6803 - accuracy: 0.6405 - val_loss: 0.6605 - val_accuracy: 0.7281
Epoch 2/8
73/73 [==============================] - 113s 2s/step - loss: 0.6502 - accuracy: 0.6776 - val_loss: 0.6309 - val_accuracy: 0.7764
Epoch 3/8
73/73 [==============================] - 103s 1s/step - loss: 0.6234 - accuracy: 0.7530 - val_loss: 0.6057 - val_accuracy: 0.8036
Epoch 4/8
73/73 [==============================] - 110s 2s/step - loss: 0.5990 - accuracy: 0.7892 - val_loss: 0.5831 - val_accuracy: 0.8187
Epoch 5/8
73/73 [==============================] - 111s 2s/step - loss: 0.5768 - accuracy: 0.8328 - val_loss: 0.5630 - val_accuracy: 0.8459
Epoch 6/8
73/73 [==============================] - 109s 1s/step - loss: 0.5563 - accuracy: 0.8534 - val_loss: 0.5439 - val_accuracy: 0.8640
Epoch 7/8
73/73 [==============================] - 103s 1s/step - loss: 0.5367 - accuracy: 0.8724 - val_loss: 0.5268 - val_accuracy: 0.8731
Epoch 8/8
73/73 [===

In [ ]:
model.save('/content/drive/MyDrive/Saved_Models/fireclassmobilev3.h5')

In [ ]:
traindata.class_indices

{'Fire': 0, 'Neutral': 1}

### Model Testing

In [ ]:
from keras.models import load_model
#from keras.preprocessing import image
import keras.utils as image
test_image=image.load_img('/content/drive/MyDrive/test/fire_images/fire.104.png',target_size=(224,224))

test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)



model=load_model('/content/drive/MyDrive/Saved_Models/fireclassmobilev3.h5')
result=model.predict(test_image)
result

if result[0][0] ==1:
  prediction='fire_images'
  print(prediction)

elif result[0][1] ==1:
  prediction='Neutral'
  print(prediction)


1/1 [==============================] - 1s 1s/step


# 5.3 InceptionResNetV2 

In [ ]:
from keras.applications.inception_resnet_v2  import InceptionResNetV2 
InceptionResNetV2_model=InceptionResNetV2(include_top=False,weights='imagenet',input_shape=(224,224,3))
InceptionResNetV2_model.summary()

219055592/219055592 [==============================] - 2s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)  

###Transfer Learning Process

In [ ]:
for layer in InceptionResNetV2_model.layers:
  layer.trainable= False


In [ ]:
X=Flatten()(InceptionResNetV2_model.output)
prediction=Dense(2,activation='softmax')(X)

In [ ]:
model=Model(inputs=InceptionResNetV2_model.input,outputs=prediction)

In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 111, 111, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                           

### Model Compiling and Training

In [ ]:
model.compile(optimizer='Adam', loss=keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [ ]:
model.fit(traindata,epochs=5,validation_data=testdata,verbose=1)

Epoch 1/5
73/73 [==============================] - 767s 10s/step - loss: 767.6813 - accuracy: 0.5625 - val_loss: 336.3391 - val_accuracy: 0.4381
Epoch 2/5
73/73 [==============================] - 749s 10s/step - loss: 125.8093 - accuracy: 0.6392 - val_loss: 353.9591 - val_accuracy: 0.3897
Epoch 3/5
73/73 [==============================] - 752s 10s/step - loss: 204.1888 - accuracy: 0.6362 - val_loss: 287.0694 - val_accuracy: 0.4622
Epoch 4/5
73/73 [==============================] - 796s 11s/step - loss: 125.0937 - accuracy: 0.6681 - val_loss: 245.1370 - val_accuracy: 0.4381
Epoch 5/5
73/73 [==============================] - 753s 10s/step - loss: 305.1973 - accuracy: 0.6022 - val_loss: 638.5246 - val_accuracy: 0.6375


In [ ]:
model.save('fireclassInceptionResNetV2_model.h5')

In [ ]:
traindata.class_indices

{'Fire': 0, 'Neutral': 1}

###Model Testing

In [ ]:
from keras.models import load_model
#from keras.preprocessing import image
import keras.utils as image
test_image=image.load_img('/content/drive/MyDrive/test/fire_images/fire.104.png',target_size=(224,224))

test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)



model=load_model('fireclassInceptionResNetV2_model.h5')
result=model.predict(test_image)
result

if result[0][0] ==1:
  prediction='fire_images'
  print(prediction)

elif result[0][1] ==1:
  prediction='non_fire_images'
  print(prediction)


1/1 [==============================] - 4s 4s/step
non_fire_images


### Model Deployment

In [ ]:
!pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 KB 20.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 7.7 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=fb524b0c8b2c46cf286e2339a9774b40914f7258f14e7b5f6ed15359043e33c2
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators


In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import cv2
from PIL import Image, ImageOps
import numpy as np
@st.cache(allow_output_mutation=True)
import streamlit as st
def load_model():
  model=tf.keras.models.load_model('/content/drive/MyDrive/Saved_Models/fireclassDense.h5')
  return model
with st.spinner('Model is being loaded..'):
  model=load_model()
 
st.write("""
         # Fire Image Classification
         """
         )
 
file = st.file_uploader("Upload the image to be classified U0001F447", type=["jpg", "png"])
st.set_option('deprecation.showfileUploaderEncoding', False)
 
def upload_predict(upload_image, model):
    
        size = (180,180)    
        image = ImageOps.fit(upload_image, size, Image.ANTIALIAS)
        image = np.asarray(image)
        img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_resize = cv2.resize(img, dsize=(224, 224),interpolation=cv2.INTER_CUBIC)
        
        img_reshape = img_resize[np.newaxis,...]
    
        prediction = model.predict(img_reshape)
        pred_class=decode_predictions(prediction,top=1)
        
        return pred_class
if file is None:
    st.text("Please upload an image file")
else:
    image = Image.open(file)
    st.image(image, use_column_width=True)
    predictions = upload_predict(image, model)
    image_class = str(predictions[0][0][1])
    score=np.round(predictions[0][0][2]) 
    st.write("The image is classified as",image_class)
    st.write("The similarity score is approximately",score)
    print("The image is classified as ",image_class, "with a similarity score of",score)

Writing app.py


In [ ]:
! streamlit run app.py & npx localtunnel –port 8501

[..................] / fetchMetadata: sill resolveWithNewModule yargs@17.1.1 ch

npx: installed 22 in 4.442s
Usage: lt --port [num] <options>

Options:
  -p, --port                Internal HTTP server port                 [required]
  -h, --host                Upstream server providing forwarding
                                             [default: "https://localtunnel.me"]
  -s, --subdomain           Request this subdomain
  -l, --local-host          Tunnel traffic to this host instead of localhost,
                            override Host header to this host
      --local-https         Tunnel traffic to a local HTTPS server     [boolean]
      --local-cert          Path to certificate PEM file for local HTTPS server
      --local-key           Path to certificate key file for local HTTPS server
      --local-ca            Path to certificate authority file for self-signed
                            certificates
      --allow-invalid-cert  Disable certificate checks for your local

### Overall Summary

Once the model is trained, it can be deployed in a production environment to classify new fire images in real time. The model can be integrated with a web or mobile application to provide alerts and recommendations based on the detected fire type, intensity, and objects involved.